<a href="https://colab.research.google.com/github/Llmortimer/character_morality/blob/main/DissertationFilmAnalyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!apt-get install -y ffmpeg

import re
import string
import pandas as pd
import numpy as np
import networkx as nx
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import matplotlib.patches as mpl
from difflib import SequenceMatcher
from operator import itemgetter



# title = "IndianaLastCrusade"
title = "TheAvengers"
# title = "StarWarsTESB"
# title = "HarryPotterDHP2" # not working script issues
# title = "TheDarkKnight"

input_script = '/content/drive/MyDrive/shared/Scripts_and_Films/' + title + '/' + title + 'Script.txt'
input_subtitles = '/content/drive/MyDrive/shared/Scripts_and_Films/' + title + '/' + title + 'Subtitles.srt'
dialogue_script = '/content/drive/MyDrive/shared/Scripts_and_Films/' + title + '/' + title + 'Dialogue.txt'
input_film = '/content/drive/MyDrive/shared/Scripts_and_Films/' + title + '/' + title + 'Film.mp4'
extracted_scenes = '/content/drive/MyDrive/shared/Scripts_and_Films/' + title + '/scenes/time.txt'

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [2]:
#pre-process subtitles by extractings start and end times of each dialogue.
f = open(input_subtitles).read()
matches = re.findall(r"([0-9]*)\n(.+)--> (.+)\n((.+\n{0,1})+)", f, re.M)
id = []
start = []
end = []
dialogue_sub = []
for i in matches:
    ex = i[3].rstrip().replace('\n', ' ')
    ex = re.sub('<[^<]+?>', '', ex)
    if input_subtitles == "/content/drive/MyDrive/shared/Scripts_and_Films/HarryPotterDHP2/HarryPotterDHP2Subtitles.srt":
        sounds = re.match(r"\([A-Z ]+\)", ex)
    else:
        sounds = re.match(r"\[[A-Z ]+\]", ex)

    names = re.match(r"[A-Z]+:", ex)
    if sounds:
        print(sounds)
        continue
    if names:
        print(names.group(0))
        ex = ex.replace(names.group(0), '')

    id.append(i[0])
    start.append(i[1])
    end.append(i[2])
    dialogue_sub.append(ex)

In [ ]:
for i in dialogue_sub:
    print(i)

In [3]:
#pre-process script by extracting the dialogue and character names writing to seperate file.
f = open(input_script).read()
matches = re.findall(r"\n\s+([A-Z ]+)\n *((.+\s)*)", f, re.M)
dialogue = []
characters = []
for i in matches:
    if '' not in i:
        j = " ".join(i[1].split())
        dialogue.append(j)
        characters.append(i[0])

        

with open(dialogue_script, "w") as ds:
    for inst in dialogue:
        ds.write(inst+"\n")

In [4]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

combined = []
last = 0

for c, i in enumerate(dialogue_sub):
    for count, j in enumerate(dialogue):

        if similar(i, j) > 0.5 and count > last and count < last + 10:
            combined.append((similar(i, j), c, count, i, j))
            last = count
            break

list1 = []
list2 = []

last_sub = -1
last_script = -1

for c in combined:
    if c[1] > last_sub+1:
        list1.append(list(range(last_sub+1, c[1])))
    elif c[1] == last_sub+1:
        list1.append([])

    list1.append([c[1]])
    last_sub = c[1]

    if c[2] > last_script+1:
        list2.append(list(range(last_script+1, c[2])))
    elif c[2] == last_script+1:
        list2.append([])

    list2.append([c[2]])
    last_script = c[2]

list1.append(list(range(last_sub+1, len(dialogue_sub))))
list2.append(list(range(last_script+1, len(dialogue))))

In [5]:
for c in combined:
    print(c)

(1.0, 15, 1, 'How bad is it?', 'How bad is it?')
(1.0, 16, 2, "That's the problem, sir. We don't know.", "That's the problem, sir. We don't know.")
(1.0, 17, 3, 'Dr. Selvig read an energy surge from the Tesseract four hours ago.', 'Dr. Selvig read an energy surge from the Tesseract four hours ago.')
(0.9347826086956522, 18, 4, "NASA didn't authorize Selvig to go to test phase.", "NASA didn't authorize Selvig to test phase.")
(0.7642276422764228, 19, 5, "He wasn't testing it. He wasn't even in the room.", "He wasn't testing it, he wasn't even in the room. Spontaneous advancement.")
(0.5882352941176471, 21, 6, '-It just turned itself on? -Where are the energy levels now?', 'It just turned itself on?')
(0.9682539682539683, 22, 8, "Climbing. When Selvig couldn't shut it down, we ordered evac.", "Climbing. When Selvig couldn't shut it down, we ordered the evac.")
(1.0, 23, 9, 'How long to get everyone out?', 'How long to get everyone out?')
(0.8674698795180723, 24, 10, 'Campus should be cle

In [6]:
char_annotated = []

for sbt, sct in zip(list1, list2):
    char = []
    dia = []
    diaSbt = []
    for i in sct:
        char.append(characters[i])
        dia.append(dialogue[i])
    
    for i in sbt:
        diaSbt.append(dialogue_sub[i])

    if len(sbt) != 0 and len(sct) != 0: 
        char_annotated.append([char, sbt, sct, diaSbt, dia, start[sbt[0]], end[sbt[-1]]])
    elif len(sbt) == 0 and len(sct) != 0:
        char_annotated.append([char, sbt, sct, ['no subtitle dialogue'], dia, 'no time info', 'no time info'])
    elif len(sct) == 0 and len(sbt) != 0 :
        char_annotated.append([['null'], sbt, sct, diaSbt, ['no script dialogue'], start[sbt[0]], end[sbt[-1]]])

In [7]:
edits = []

for count, dialogue in enumerate(char_annotated):
    #where the speaker of the dialogue is set to 'null'
    if dialogue[0] == ['null']:
        #we only consider the subtitle dialogues to move to other peices of dialogue as monologues exist within the script
        query_dialogues = dialogue[3]
        #for every line in the script dialogues
        for i, query_dialogue in enumerate(query_dialogues):
            #check if it is present in the previous or the post dialogues
            if query_dialogue.translate(str.maketrans('', '', string.punctuation)) in char_annotated[count-1][4][0].translate(str.maketrans('', '', string.punctuation)) and query_dialogue not in char_annotated[count-1][3][0]:
                #if its present in the previous script dialogue but not in the subtitles dialogue: add details to a list of edits to do
                edits.append((count, count-1, dialogue[1][i], len(query_dialogues)-i, query_dialogue))

            elif query_dialogue in char_annotated[count+1][4][0] and query_dialogue not in char_annotated[count+1][3][0]:
                #if it is present in the next script dialogue but not in the subtitles dialogue: add details to a list of edits to do
                edits.append((count, count+1, dialogue[1][i], len(query_dialogues)-i, query_dialogue))


In [ ]:
for c in edits:
    print(c)

In [8]:
#in the list edits there contains: (index of line in char_annotated to be moved, index of what line it should be in, exact id of the line to be moved, if it is moved forwards or backwards, the line to be moved)
for e in edits:
    if e[0] > e[1]:
        #append the literal line to the previous one 
        char_annotated[e[1]][3].append(e[4])
        #edit the end time of the appended line
        char_annotated[e[1]][6] = end[e[2]]
        #append the id of the line to the previous one
        char_annotated[e[1]][1].append(e[2])
    else:
        #insert the literal line to the beginning previous one 
        char_annotated[e[1]][3].insert(0, e[4])
        #edit the end time of the inserted line
        char_annotated[e[1]][6] = start[e[2]]
        #insert the id of the line to the previous one
        char_annotated[e[1]][1].insert(0, e[2])

i = 0
for e in edits:
    if e[3] == 1:
        del char_annotated[e[0]-i]
        i+=1

In [ ]:
for i in char_annotated:
    print(i)

In [9]:
to_delete = []

for count, i in enumerate(char_annotated):
    if i[0] == ['null']:
        to_delete.append(count)

    if i[5] == 'no time info' and i[6] == 'no time info':
        to_delete.append(count)

    

for count, i in enumerate(to_delete):
    del char_annotated[i-count]

In [10]:
final_dialogue = []

for i in char_annotated:
    for count, j in enumerate(i[0]):
        temp, ms = i[5].split(',')
        h, m, s = temp.split(':')
        start_time = int(h) * 3600 + int(m) * 60 + int(s) + float(ms)/1000
        temp, ms = i[6].split(',')
        h, m, s = temp.split(':')
        end_time = int(h) * 3600 + int(m) * 60 + int(s) + float(ms)/1000

        final_dialogue.append([j, i[4][count], start_time, end_time])


In [11]:
for f in final_dialogue:
    print(f)

[' ', '"And there came a day, a day unlike any other, when Earth\'s mightiest heroes and heroines found themselves united against a common threat. On that day, the Avengers were born--to fight the foes no single superhero could withstand! Through the years, their roster has prospered, changing many times, but their glory has never been denied! Heed the call, then--for now, the Avengers Assemble!"', 27.11, 98.014]
['NICK FURY', 'How bad is it?', 120.495, 121.538]
['AGENT PHIL COULSON', "That's the problem, sir. We don't know.", 122.205, 124.791]
['AGENT PHIL COULSON', 'Dr. Selvig read an energy surge from the Tesseract four hours ago.', 129.129, 132.006]
['NICK FURY', "NASA didn't authorize Selvig to test phase.", 132.09, 134.05]
['AGENT PHIL COULSON', "He wasn't testing it, he wasn't even in the room. Spontaneous advancement.", 134.134, 136.219]
['AGENT MARIA HILL', 'It just turned itself on?', 137.72, 140.181]
['AGENT PHIL COULSON', "Climbing. When Selvig couldn't shut it down, we ord

In [ ]:
#scene detection, for each frame assign a probablility that the scene has changed given the change of intensity of the pixels on the pervious frame.
#if this probablility is higher than 0.5 for a given frame the frame time, and probablilty have been output to a file "time.txt" and the frame itself output to a PNG in scenes/ 

!ffmpeg -i /content/drive/MyDrive/shared/Scripts_and_Films/TheAvengers/TheAvengersFilm.mp4 -filter_complex "select='gt(scene,0.5)',metadata=print:file=/content/drive/MyDrive/shared/Scripts_and_Films/TheAvengers/scenes/time.txt" -vsync vfr /content/drive/MyDrive/shared/Scripts_and_Films/TheAvengers/scenes/img%03d.png

In [12]:
#extract the scene changes and their times from the text file generated by the scene change algorithm

f = open(extracted_scenes).read()
matches = re.findall(r"frame:([0-9]+).+pts_time:([0-9.]+)", f)

scenes = {}
previous = '0'

for match in matches:
    scenes[match[0]] = {}
    scenes[match[0]]['start'] = float(previous)
    scenes[match[0]]['end'] = float(match[1])
    previous = match[1]

In [78]:
print(scenes)

{'0': {'start': 0.0, 'end': 71.7801}, '1': {'start': 71.7801, 'end': 256.757}, '2': {'start': 256.757, 'end': 258.634}, '3': {'start': 258.634, 'end': 270.562}, '4': {'start': 270.562, 'end': 272.689}, '5': {'start': 272.689, 'end': 275.817}, '6': {'start': 275.817, 'end': 288.413}, '7': {'start': 288.413, 'end': 295.796}, '8': {'start': 295.796, 'end': 339.214}, '9': {'start': 339.214, 'end': 339.631}, '10': {'start': 339.631, 'end': 339.756}, '11': {'start': 339.756, 'end': 344.678}, '12': {'start': 344.678, 'end': 345.345}, '13': {'start': 345.345, 'end': 512.846}, '14': {'start': 512.846, 'end': 532.741}, '15': {'start': 532.741, 'end': 532.824}, '16': {'start': 532.824, 'end': 533.116}, '17': {'start': 533.116, 'end': 543.502}, '18': {'start': 543.502, 'end': 545.879}, '19': {'start': 545.879, 'end': 570.112}, '20': {'start': 570.112, 'end': 573.198}, '21': {'start': 573.198, 'end': 574.408}, '22': {'start': 574.408, 'end': 579.496}, '23': {'start': 579.496, 'end': 592.551}, '24':

In [16]:
# eliminate all characters who only speak once
to_eliminate = {}

for c in characters:
    to_eliminate[c] = 0

print(to_eliminate)

for f in final_dialogue:
    to_eliminate[f[0]] += 1

print(to_eliminate)

eliminate = []
for c, i in enumerate(to_eliminate):
    if to_eliminate[i] <= 3:
        eliminate.append(i)

print(eliminate)

to_elim = []
for c in eliminate:
    for i, f in enumerate(final_dialogue):
        if f[0] == c:
            to_elim.append(i)

for c, f in enumerate(to_elim):
    del final_dialogue[f-c]

{' ': 0, 'NICK FURY': 0, 'AGENT PHIL COULSON': 0, 'AGENT MARIA HILL': 0, 'SELVIG': 0, 'CLINT BARTON': 0, 'LOKI': 0, 'BACK AT THE LAB': 0, 'A CLOUD OF BLUE LIGHT CONSUMES THE ENTIRE FACILITY AND PARTS OF': 0, 'LUCHKOV': 0, 'NATASHA': 0, 'WEASELLY THUG': 0, 'ATTENDING WOMAN': 0, 'LITTLE GIRL': 0, 'BANNER': 0, 'SHIELD SCIENTIST': 0, 'STEVE': 0, 'TONY': 0, 'PEPPER POTTS': 0, 'PEPPER': 0, 'JARVIS': 0, 'PILOT': 0, 'THE OTHER': 0, 'LIFT FANS MOUNTED ON THE SIDES STARTS TO LIFT INTO THE AIR IN': 0, 'AGENT JASPER SITWELL': 0, 'ELDER GERMAN MAN': 0, 'CAPTAIN AMERICA': 0, 'IRON MAN': 0, 'THOR': 0, 'INTERCUTS': 0, 'STEVE TONY': 0, 'NATASHA BANNER': 0, 'TONY STEVE': 0, 'SHIELD BASE VOICE': 0, 'BARTON': 0, 'OUTSIDE THE HELICARRIER': 0, 'GALAGA PLAYER': 0, 'HELMSMAN': 0, 'INSIDE THE SUIT': 0, 'SECURITY GUARD': 0, 'MONTAGE': 0, 'YOUNG SHIELD PILOT': 0, 'HAWKEYE': 0, 'BLACK WIDOW': 0, 'YOUNG COP': 0, 'POLICE SERGEANT': 0, 'BREAKS OFF A MASSIVE PIECE OF ARMOR AND SLAMS IT DOWN ONTO THE': 0, 'SENATOR BOY

In [17]:
scene_dialogues = []
character_list = []
scene_combined = {}

for s in scenes:
    for d in final_dialogue:
        if d[2] > scenes[s]['start'] and d[3] < scenes[s]['end']:
            print((s,d))
            scene_dialogues.append((s, d))

for i, j in scene_dialogues:
    scene_combined[i] = []

for i, j in scene_dialogues:
    if j[0] not in character_list:
        character_list.append(j[0])

    scene_combined[i].append(j)

print(scene_combined)
   

('1', ['NICK FURY', 'How bad is it?', 120.495, 121.538])
('1', ['AGENT PHIL COULSON', "That's the problem, sir. We don't know.", 122.205, 124.791])
('1', ['AGENT PHIL COULSON', 'Dr. Selvig read an energy surge from the Tesseract four hours ago.', 129.129, 132.006])
('1', ['NICK FURY', "NASA didn't authorize Selvig to test phase.", 132.09, 134.05])
('1', ['AGENT PHIL COULSON', "He wasn't testing it, he wasn't even in the room. Spontaneous advancement.", 134.134, 136.219])
('1', ['AGENT MARIA HILL', 'It just turned itself on?', 137.72, 140.181])
('1', ['AGENT PHIL COULSON', "Climbing. When Selvig couldn't shut it down, we ordered the evac.", 140.265, 142.684])
('1', ['NICK FURY', 'How long to get everyone out?', 142.767, 143.977])
('1', ['AGENT PHIL COULSON', 'Campus should be clear in the next half hour.', 144.06, 145.728])
('1', ['NICK FURY', 'Do better.', 145.812, 146.729])
('1', ['AGENT MARIA HILL', 'Sir, evacuation may be futile.', 147.897, 149.899])
('1', ['NICK FURY', 'We should t

In [ ]:
for s in scene_combined:
    print(scene_combined[s])

In [18]:
graph_matrix = np.zeros((len(character_list), len(character_list)))
current_scene = 0
scene_characters = []
scene_durations = []

for s, d in scene_dialogues:  
    if d[0] not in scene_characters:
        scene_characters.append(d[0])

    #work out a way to calculate weights for each scene individually, sum of total time for each character?

weightings_out = {}
scene_weightings = {}

for scene in scene_combined:
    for dialogue in scene_combined[scene]:
        if dialogue[0] not in weightings_out:
            weightings_out[dialogue[0]] = 0

        weightings_out[dialogue[0]] = weightings_out[dialogue[0]] + round(d[3] - d[2], 3)

    scene_weightings[scene] = weightings_out
    weightings_out = {}

print(scene_weightings)
longest_dialogue = 0

for i in scene_weightings:
    for j in scene_weightings[i]:
        if scene_weightings[i][j] > longest_dialogue:
            longest_dialogue = scene_weightings[i][j]

{'1': {'NICK FURY': 29.07000000000001, 'AGENT PHIL COULSON': 8.55, 'AGENT MARIA HILL': 8.55, 'SELVIG': 10.260000000000002, 'CLINT BARTON': 3.42}, '13': {'LOKI': 11.970000000000002, 'SELVIG': 3.42, 'NICK FURY': 11.970000000000002, 'CLINT BARTON': 5.13, 'AGENT MARIA HILL': 1.71}, '28': {'NICK FURY': 17.100000000000005, 'AGENT MARIA HILL': 1.71, 'LUCHKOV': 10.260000000000002, 'NATASHA': 46.170000000000016, 'AGENT PHIL COULSON': 11.970000000000002, 'BANNER': 34.20000000000001}, '40': {'STEVE': 1.71, 'TONY': 5.13, 'PEPPER': 3.42}, '44': {'JARVIS': 3.42, 'TONY': 42.750000000000014, 'PEPPER': 39.33000000000001, 'AGENT PHIL COULSON': 6.84}, '45': {'STEVE': 3.42, 'AGENT PHIL COULSON': 5.13}, '50': {'STEVE': 3.42, 'AGENT PHIL COULSON': 1.71}, '54': {'THE OTHER': 6.84, 'LOKI': 8.55}, '59': {'AGENT PHIL COULSON': 3.42, 'STEVE': 1.71, 'NATASHA': 1.71}, '61': {'STEVE': 1.71}, '63': {'BANNER': 1.71, 'STEVE': 1.71}, '64': {'BANNER': 1.71}, '66': {'BANNER': 1.71}, '70': {'BANNER': 1.71}, '82': {'AGENT 

In [17]:
from networkx.algorithms.community.centrality import girvan_newman
import itertools

comp = girvan_newman(G)
k = 10
for communities in itertools.islice(comp, k):
    print(tuple(sorted(c) for c in communities))


o = tuple(sorted(c) for c in next(comp))
print(o)

NameError: ignored

In [20]:
#collect durations into the graph matrix
idx_in = []
for scene in scene_weightings:
    for char in scene_weightings[scene]:
        idx_out = scene_characters.index(char)
        for listener in scene_weightings[scene]:
            idx_in.append(scene_characters.index(listener))

        # print(idx_out)
        # print(idx_in)

        for idx in idx_in:
            graph_matrix[idx_out][idx] += scene_weightings[scene][char]

        idx_in = []

df = pd.DataFrame(graph_matrix)
df = (df).round(3)
df.columns = scene_characters
df.index = scene_characters
# print(df)


#generate the graph for degree centrality including all characters
plt.figure(figsize=(20,12))

G = nx.from_pandas_adjacency(df, nx.DiGraph)
weight = nx.get_edge_attributes(G, "weight")
pos = nx.spring_layout(G)
nx.set_node_attributes(G, pos, "pos")

weight_list = [f'{w}' for w in list(nx.get_edge_attributes(G, 'weight').values())]

edge_x = []
edge_y = []
xtext = []
ytext = []
for edge in G.edges():
    x0, y0 = G.nodes[edge[0]]['pos']
    x1, y1 = G.nodes[edge[1]]['pos']
    xtext.append((x0+x1)/2)
    ytext.append((y0+y1)/2)
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

# for i in weight:
#     weight_list.append(weight[i])

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    # hoverinfo='none',
    text=weight_list,
    hoverinfo='text',
    mode='lines')


eweights_trace = go.Scatter(x=xtext,y= ytext, mode='text',
    marker_size=0.5,
    text=weight_list,
    textposition='top center',
    hovertemplate='%{text}')


node_x = []
node_y = []
for node in G.nodes():
    x, y = G.nodes[node]['pos']
    node_x.append(x)
    node_y.append(y)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        # colorscale options
        #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
        #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
        #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
        colorscale='Reds',
        # reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line_width=2))

node_adjacencies = []
node_text = []
degree_centrality = []
for node, adjacencies in enumerate(G.adjacency()):
    node_adjacencies.append(len(adjacencies[1]))
    node_text.append(scene_characters[node]+', # of connections: '+str(len(adjacencies[1])))
    degree_centrality.append(len(adjacencies[1]))

# dc = pd.DataFrame(degree_centrality)
node_trace.marker.color = node_adjacencies
node_trace.text = node_text

fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='Graph Showing Degree Centrality of Each Node',
                titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                # annotations=[ dict(
                #     text="Python code: <a href='https://plotly.com/ipython-notebooks/network-graphs/'> https://plotly.com/ipython-notebooks/network-graphs/</a>",
                #     showarrow=False,
                #     xref="paper", yref="paper",
                #     x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )
fig.show()

<Figure size 1440x864 with 0 Axes>

In [19]:
import plotly.express as px

fig = px.scatter(x=scene_characters, y=degree_centrality)
fig.update_xaxes(categoryorder='total descending')
fig.show()

In [ ]:
#generate df for network not including indy
adj_mat_noIndy = np.zeros((len(character_list)-1, len(character_list)-1))
idx_in = []
for scene in scene_weightings:
    for char in scene_weightings[scene]:
        if char == 'INDY':
            continue

        idx_out = scene_characters.index(char)
        if idx_out > 8:
            idx_out -= 1

        for listener in scene_weightings[scene]:
            if listener == 'INDY':
                continue
            idx_in_temp = scene_characters.index(listener)
            if idx_in_temp > 8:
                idx_in_temp -= 1

            idx_in.append(idx_in_temp)

        for idx in idx_in:
            adj_mat_noIndy[idx_out][idx] += scene_weightings[scene][char]

        idx_in = []


df_noIndy = pd.DataFrame(adj_mat_noIndy)
df_noIndy = df_noIndy.round(3)
df_noIndy.columns = [x for i,x in enumerate(scene_characters) if x!='INDY']
df_noIndy.index = [x for i,x in enumerate(scene_characters) if x!='INDY']


G = nx.from_pandas_adjacency(df_noIndy, nx.DiGraph)
weight = nx.get_edge_attributes(G, "weight")
pos = nx.spring_layout(G)
nx.set_node_attributes(G, pos, "pos")

edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = G.nodes[edge[0]]['pos']
    x1, y1 = G.nodes[edge[1]]['pos']
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

weight_list = []
for i in weight:
    weight_list.append(weight[i])

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

node_x = []
node_y = []
for node in G.nodes():
    x, y = G.nodes[node]['pos']
    node_x.append(x)
    node_y.append(y)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        # colorscale options
        #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
        #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
        #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
        colorscale='Reds',
        # reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line_width=2))

node_adjacencies = []
node_text = []
for node, adjacencies in enumerate(G.adjacency()):
    node_adjacencies.append(len(adjacencies[1]))
    if node >= 8:
        node+=1
    node_text.append(scene_characters[node]+', # of connections: '+str(len(adjacencies[1])))

node_trace.marker.color = node_adjacencies
node_trace.text = node_text

fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='Graph Showing Degree Centrality of Each Node without Indy',
                titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                # annotations=[ dict(
                #     text="Python code: <a href='https://plotly.com/ipython-notebooks/network-graphs/'> https://plotly.com/ipython-notebooks/network-graphs/</a>",
                #     showarrow=False,
                #     xref="paper", yref="paper",
                #     x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )
fig.show()

In [ ]:
from networkx.algorithms.community.centrality import girvan_newman
from networkx.algorithms.community import k_clique_communities
import itertools

comp = girvan_newman(G)
k = 5
for communities in itertools.islice(comp, k):
    print(tuple(sorted(c) for c in communities))


K5 = nx.convert_node_labels_to_integers(G, first_label=2)
G.add_edges_from(K5.edges())
c = list(k_clique_communities(G, 4))

(['OFFICER'], ['FIRST SCOUT'], ['SCOUTMASTER'], ['FEDORA', 'HALFBREED', 'HERMAN', 'MAN', 'PANAMA HAT', 'ROSCOE', 'ROUGH RIDER', 'SHERIFF'], ['BRODY', 'BUTLER', 'DONOVAN', 'ELSA', 'HENRY', 'KNIGHT', 'MAN IN CHAIR', 'NAZI SOLDIER', 'SALLAH', 'SECOND MAN', 'SULTAN', 'VOGEL'], ['IRENE', 'MALE STUDENT'], ['KAZIM'], ['WOMAN'])
(['OFFICER'], ['FIRST SCOUT'], ['SCOUTMASTER'], ['FEDORA', 'HALFBREED', 'HERMAN', 'MAN', 'PANAMA HAT', 'ROSCOE', 'ROUGH RIDER', 'SHERIFF'], ['BRODY', 'BUTLER', 'DONOVAN', 'ELSA', 'HENRY', 'KNIGHT', 'MAN IN CHAIR', 'NAZI SOLDIER', 'SALLAH', 'SECOND MAN', 'VOGEL'], ['IRENE', 'MALE STUDENT'], ['KAZIM'], ['WOMAN'], ['SULTAN'])
(['OFFICER'], ['FIRST SCOUT'], ['SCOUTMASTER'], ['FEDORA', 'HALFBREED', 'HERMAN', 'MAN', 'PANAMA HAT', 'ROSCOE', 'ROUGH RIDER', 'SHERIFF'], ['BRODY', 'DONOVAN', 'ELSA', 'HENRY', 'KNIGHT', 'MAN IN CHAIR', 'NAZI SOLDIER', 'SALLAH', 'SECOND MAN', 'VOGEL'], ['IRENE', 'MALE STUDENT'], ['KAZIM'], ['BUTLER'], ['WOMAN'], ['SULTAN'])
(['OFFICER'], ['FIRST SCO

NetworkXNotImplemented: ignored